In [1]:
%matplotlib inline
import os, flopy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyvista as pv
import vtk
from pyproj import Transformer

flopy is installed in /home/wk-gida1/.local/lib/python3.7/site-packages/flopy


---

### Geotransformation of well location

In [2]:
wellLoc = pd.read_csv('../inputData/TV-HFM_Wells_1Location.csv',header=1,index_col=0,
                      usecols=[0,2,3,6],skipfooter=1,engine='python')
transformer = Transformer.from_crs('esri:102605','epsg:32611',always_xy=True)
points = list(zip(wellLoc.Easting,wellLoc.Northing))
coordsWgsUTM = np.array(list(transformer.itransform(points)))
wellLoc['EastingUTM']=coordsWgsUTM[:,0]
wellLoc['NorthingUTM']=coordsWgsUTM[:,1]
wellLoc['Elevation_m']=wellLoc['Altitude_ft']*0.3048
wellLoc.to_csv('../inputData/TV-HFM_Wells_1Location_Wgs11N.csv')

---
### Transformation of lithology elevation

In [3]:
wellLito = pd.read_csv('../inputData/TV-HFM_Wells_2Lithology.csv',skipfooter=1,
                       header=1,usecols=[0,1,2,3],engine='python')
wellLito["topLitoElev_m"] = ''
wellLito["botLitoElev_m"] = ''
wellLito.head()    

Bore  Depth_top_L  Depth_bot_L PrimaryLith topLitoElev_m botLitoElev_m
0  A. Isaac          0.0          1.0        Soil                            
1  A. Isaac          1.0         53.0        Sand                            
2  A. Isaac         53.0        248.0      Basalt                            
3  A. Isaac        248.0        265.0        Sand                            
4  A. Isaac        265.0        323.0      Basalt

In [4]:
for index, values in wellLito.iterrows():
    x, y, z =wellLoc.loc[values.Bore][['EastingUTM','NorthingUTM','Elevation_m']]
    topLito = z - (values.Depth_top_L)*0.3048 
    botLito = z- (values.Depth_bot_L)*0.3048 
    wellLito.loc[index,"topLitoElev_m"]=topLito
    wellLito.loc[index,"botLitoElev_m"]=botLito
wellLito.set_index('Bore', inplace=True)

In [5]:
wellLito.to_csv('../inputData/TV-HFM_Wells_2Lithology_m.csv')
wellLito.head()

Depth_top_L  Depth_bot_L PrimaryLith topLitoElev_m botLitoElev_m
Bore                                                                      
A. Isaac          0.0          1.0        Soil       976.579       976.274
A. Isaac          1.0         53.0        Sand       976.274       960.425
A. Isaac         53.0        248.0      Basalt       960.425       900.989
A. Isaac        248.0        265.0        Sand       900.989       895.807
A. Isaac        265.0        323.0      Basalt       895.807       878.129

In [ ]:
#generation of list arrays for the vtk
offsetList = []
linSec = []
linVerts = []

i=0
for index, values in wellLito.iterrows():
    x, y, z =wellLoc.loc[index][['EastingUTM','NorthingUTM','Elevation_m']]
    topLito = z - (values.Depth_top_L)*0.3048 
    botLito = z- (values.Depth_bot_L)*0.3048 
    cellVerts = [[x,y,topLito],[x,y,botLito]]
    offsetList.append(i*3)         
    linSec = linSec + [2,2*i,2*i+1]
    linVerts = linVerts + cellVerts
    i +=1

offsetArray = np.array(offsetList)
linArray = np.array(linSec)
cellType = np.ones([i])*3
vertArray = np.array(linVerts)
# create the unstructured grid and assign lito code
grid = pv.UnstructuredGrid(offsetArray, linArray, cellType, vertArray)
grid.cell_arrays["values"] = wellLito.litoCode.values
grid.save('../outputData/wellLito.vtu',binary=False)